In [ ]:
import pandas as pd
import requests
import json
import time
import random
import base64
from IPython.display import display, HTML
from datetime import date
import numpy as np
from pandas.io.json import json_normalize
import re
import os

In [11]:
brand_tag = "Vivienne_Westwood"
# This has to be the Poshmark URL version of the brand name, also know as the brand tag.
# You can find the Poshmark brand_tag of the brand name by searching Poshmark for the brand here:
# https://poshmark.com/brands/
# Then when clicking on the brand, you can find the brand_tag at the end of the link:
# https://poshmark.com/brand/Vivienne_Westwood
# Vivienne Westwood = Vivienne_Westwood

In [ ]:
def PoshmarkBrandScraper(brand_tag):

  # pd.set_option('display.max_columns', 100)

  url_start = "https://poshmark.com"
  url_2 = "/vm-rest/channel_groups/brand/channels/" + brand_tag
  url_postrequest = "/collections/post?request={%22filters%22:"
  url_json_start = "{%22brand%22:[%22" + brand_tag + "%22],"
  url_inventory = "%22inventory_status%22:[%22" + "all" + "%22]},"
  url_facets = "%22facets%22:[%22color%22,%22department%22],%22experience%22:%22all%22,%22sizeSystem%22:%22us%22,"
  url_tail = "%22count%22:%2248%22}&summarize=true&pm_version=209.0.0"

  start_url = url_start + url_2 + url_postrequest + url_json_start + url_inventory + url_facets + url_tail

  MIN_SLEEP_TIME = 2
  page = 1


  def get_table_from_url(url):
    payload={}
    headers = {
      'authority': 'poshmark.com',
      'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
      'accept-language': 'en-US,en;q=0.9,ja;q=0.8',
      'cache-control': 'max-age=0',
      'if-none-match': 'W/"98bcb-l0mGc5jkSNCN+QgJpGNYW261QjY"',
      'sec-ch-ua': '"Chromium";v="104", " Not A;Brand";v="99", "Google Chrome";v="104"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"macOS"',
      'sec-fetch-dest': 'document',
      'sec-fetch-mode': 'navigate',
      'sec-fetch-site': 'none',
      'sec-fetch-user': '?1',
      'upgrade-insecure-requests': '1',
      'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    # print(response.text)

    # print("url: ",url)
    # print("status: ",response.status_code)
    # print("text: ",response.text)
    # print("content: ", response.content)

    if response.status_code != 200:
        print("Couldn't Connect to Poshmark")
    else:
        # print("success")
        text = response.text
        nested_text = json.loads(text)
        # print(nested_text)
        if "statusCode" in nested_text:
            if nested_text["statusCode"] == 404:
                print("Couldn't find Poshmark Brand")
        elif "data" in nested_text:
            df = pd.json_normalize(nested_text['data'])
            # total_items = more_data['total']
            try:
                if 'more' in nested_text:
                    more_data = nested_text['more']
                    if "next_max_id" in more_data:
                        NewMaxID = str(more_data['next_max_id'])
                        url_nextmax = "%22max_id%22:%22" + NewMaxID + "%22,"
                        new_url = url_start + url_2 + url_postrequest + url_json_start + url_inventory + url_facets + url_nextmax + url_tail
                    else:
                        new_url = "end"
                else:
                    new_url = "end"
            except:
                new_url = "end"

            return df, new_url
  try:
    brand_df, next_url = get_table_from_url(start_url)
    if brand_df.size > 0:
      # print("Table Size: ",brand_df.size)
      while next_url != "end":
          print(" Scraping page: ", str(page), " of ", brand_tag, ". ", str(brand_df.shape[0]), ' items information collected')
          table_to_add, next_url = get_table_from_url(next_url)
          brand_df = pd.concat([brand_df, table_to_add])
          sleep_time = MIN_SLEEP_TIME + random.random() * 10
          time.sleep(int(sleep_time))
          page = page + 1
      # print(brand_df.head(10))
      return brand_df
    else:
      print("No brand data collected")
      return None
  except Exception as e: print(e)

  

brand_info = PoshmarkBrandScraper(brand_tag)#


In [33]:
from datetime import datetime
from google.colab import drive, files
from urllib.request import urlopen
from urllib.parse import urlparse
drive.flush_and_unmount()
drive.mount('/content/drive')

def download_csv_to_google_drive(df):
  output_directory = "/content/drive/My Drive/Poshmark_Brands"
  user_dir = output_directory + '/' + brand_tag
  now = datetime.now()
  date_time = now.strftime("%Y%d%m_%H%M%S")
  csv_filename = user_dir + "/" + brand_tag + ".csv"
  if not os.path.isfile(user_dir):
    os.makedirs(user_dir, exist_ok=True)
  df.to_csv(csv_filename,index=False)
  print("Saved to Google Drive")
try:
    download_csv_to_google_drive(df=brand_info)
    drive.flush_and_unmount()

except Exception as e: 
    print(e)
    drive.flush_and_unmount()

Mounted at /content/drive
Saved to Google Drive


In [ ]:
# Option to upload csv if you already downloaded the scraped data to your desktop and want to run analysis without rescraping:
# ONLY WORKS WITH SMALL SIZED CSV FILES LESS THAN 10MB
# IF UPLOAD A LARGE CSV FILE, USE GOOGLE DRIVE IMPORT
from google.colab import files
import io
uploaded = files.upload()
brand_info = pd.read_csv(io.BytesIO(uploaded[f"{brand_tag}.csv"]))
print(brand_info.head(5))

In [ ]:
# Option to upload csv directly from Google Drive if you have is saved in a folder
# USED FOR LARGE CSV FILES
from google.colab import auth
auth.authenticate_user()
import pandas as pd
from google.auth import default
creds, _ = default()

# insert url link to public csv file in good drive
link = 'https://drive.google.com/file/d/1-9epCNa44aT8xPGr4PV9F7KsGbHg5Y9V/view'
 
# to get the id part of the file
id = link.split("/")[-2]
print(id)
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile(f"{brand_tag}.csv") 
 
brand_info = pd.read_csv(f"{brand_tag}.csv")
print(brand_info.head(5))

In [38]:
# VIEW TOP MOST 10 ITEMS IN THE DATAFRAME - USED TO CONFIRM SUCCESSFUL DATAFRAME CREATION
brand_info_reduced = brand_info[['title', 'creator_username', 'price', 'inventory.status','share_count', 'like_count','brand','catalog.department_obj.slug','category','category_v2.slug','first_available_at','status_changed_at']]
print(brand_info_reduced.head(10))

                                               title creator_username  price  \
0        Vivienne Westwood Man 3Button Collar Shirt…    sauvagebeauty    175   
1                 NIB VIVIENNE WESTWOOD PURPLE PUMPS         joydubay     89   
2  Vivienne Westwood Wallet Purse Orb Grey Black ...       bagzposher    603   
3     VIVIENNE WESTWOOD Anglomania LS Blouse Size 42    sauvagebeauty    166   
4                           Worlds End John Bull hat  wardrobefantasy    206   
5  Vivienne Westwood Orb Slip On Loafers Mens 40 ...     emersyn_rose    149   
6        Vivienne Westwood Mens Wool Jacket Size 52…    sauvagebeauty   1103   
7                         VIVIENNE WESTWOOD Culottes    sauvagebeauty    350   
8  Vivienne Westwood gray wool beret with embroid...         delyn123    115   
9   AUTH Vivienne Westwood Piercing Orb Silver Woman     bismaskloset    873   

  inventory.status  share_count  like_count              brand  \
0         sold_out           13           5  Vivienne

In [39]:
# VIEW TOP 20 SOLD ITEMS SORTED BY PRICE
sales = brand_info_reduced[brand_info['inventory.status'] == "sold_out"]
sales = sales.sort_values(by=['price'], ascending=False)
print(sales[['title','price','category_v2.slug','status_changed_at']].head(20))

                                                title  price category_v2.slug  \
22  Vivienne Westwood F/W 1992 Rare Always On Came...   2700             Tops   
11  Vivienne Westwood F/W 1992 Rare Always On Came...   2700             Tops   
23  Vivienne Westwood Spring 1994 Café Society Leo...   1800             Tops   
8         NWT Vivienne Westwood Dame Corset Dress 8UK   1500          Dresses   
12                       Vivienne Westwood corset top   1440             Tops   
41  Vivienne Westwood Wool Ruffled Panels Skirt w/...   1200           Skirts   
23   Beautiful Vivienne Westwood black feather corset   1200             Tops   
16          Vivienne Westwood NWT Archive Orb Yasmine   1200             Bags   
37                      Rare Vivienne Westwood corset   1200             Tops   
31                Vivienne Westwood 1997 draped dress   1100          Dresses   
1         Vintage 1981 Original Westwood Pirate Shirt   1100           Shirts   
23  Vivienne Westwood Corset